In [1]:
import pandas as pd

In [2]:
data =pd.read_csv("C:\\Users\skrPlays\Downloads\\russia_tweets.csv", names= ["tweetcreatedts","extractedts","userid","tweetid","text","language","formatted_dt"])
data['tweet_date'] =  pd.to_datetime(data['tweetcreatedts'],  errors='coerce', infer_datetime_format=True).dt.date
data.dropna(subset=['tweet_date'], inplace=True)

In [ ]:
data.head()

In [4]:
data.isnull().sum()

tweetcreatedts    0
extractedts       0
userid            0
tweetid           0
text              0
language          0
formatted_dt      0
tweet_date        0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242038 entries, 918 to 344356
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tweetcreatedts  242038 non-null  object
 1   extractedts     242038 non-null  object
 2   userid          242038 non-null  int64 
 3   tweetid         242038 non-null  int64 
 4   text            242038 non-null  object
 5   language        242038 non-null  object
 6   formatted_dt    242038 non-null  object
 7   tweet_date      242038 non-null  object
dtypes: int64(2), object(6)
memory usage: 16.6+ MB


In [9]:
data['tweet_date'].value_counts()

2022-04-08    11363
2022-04-04     6297
2022-04-03     6257
2022-04-02     6115
2022-03-05     5621
2022-03-23     5609
2022-04-09     5604
2022-03-21     5439
2022-03-03     5349
2022-03-06     5257
2022-04-05     5234
2022-04-20     4616
2022-04-16     4550
2022-03-13     4459
2022-03-30     4334
2022-03-07     4321
2022-04-19     4286
2022-04-23     4227
2022-04-07     4209
2022-04-01     4151
2022-03-31     4118
2022-03-18     4037
2022-04-27     3990
2022-03-09     3979
2022-04-11     3975
2022-03-04     3929
2022-04-06     3857
2022-03-02     3848
2022-04-17     3847
2022-03-01     3776
2022-03-25     3771
2022-03-29     3660
2022-03-19     3633
2022-03-20     3607
2022-04-18     3580
2022-03-28     3561
2022-03-27     3470
2022-03-12     3437
2022-04-15     3398
2022-04-25     3361
2022-04-12     3318
2022-04-10     3306
2022-03-24     3303
2022-03-08     3290
2022-02-28     3288
2022-04-22     3268
2022-04-13     3199
2022-03-10     3193
2022-04-24     3185
2022-03-17     3178


In [8]:
len(data['tweet_date'].value_counts())

60

In [10]:
(data.groupby('tweet_date').head(1000).reset_index(drop=True)).to_csv('russian_tweets1000s.csv',index=False)

#### From here, put the code in Google collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/

In [ ]:
import pandas as pd
import numpy as np
import timeit
!pip install deep-translator
from deep_translator import GoogleTranslator

In [ ]:
df =pd.read_csv("/content/drive/MyDrive/Colab Notebooks/russian_tweets1000s.csv")

In [ ]:
df.shape

In [ ]:
df.dropna(subset=['text'], inplace=True)
print(df.shape)
print(df['language'].value_counts())

In [ ]:
print(min(df['tweet_date']),max(df['tweet_date']))

In [ ]:
import re
def preprocess_uk(sentence):
    sentence=str(sentence)
    sentence =  re.sub(r'()#\w+', r'\1', sentence) #removes hashtags
    sentence = re.sub(r'@\w+', '', sentence)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",sentence).split())

df['processed_text']=df['text'].map(lambda s:preprocess_uk(s))

In [ ]:
df_ru_split = np.array_split(df, 25)
print(df_ru_split[0].shape,df_ru_split[1].shape)

In [ ]:
df['text']

In [ ]:
df['processed_text']

In [ ]:
df['processed_text'].isnull().sum()

In [ ]:
max(df_ru_split[10]['tweetcreatedts'])

In [ ]:
import time

# for i in range(1,3):
#   arr_df_it.append( pd.DataFrame())
df_trans_ru = pd.DataFrame()
for i in range(len(df_ru_split)):
    print(i)
    start_time = timeit.default_timer()
    df_ru_split[i]['translated']=GoogleTranslator(source='ru', target='en').translate_batch(df_ru_split[i]['processed_text'].to_list())
    print(timeit.default_timer() - start_time)
    df_trans_ru = pd.concat([df_trans_ru,df_ru_split[i]],axis=0)
    if (i%5==0 and i!=0) or (i==len(df_ru_split)-1):
      df_trans_ru.to_csv('df_trans_ru'+str(i)+'.csv',index=False)

    time.sleep(1)